#### 문제 설명  
n개의 점으로 이루어진 트리가 있습니다. 이때, 트리 상에서 다음과 같은 것들을 정의합니다.  
  
- 어떤 두 점 사이의 거리는, 두 점을 잇는 경로 상 간선의 개수로 정의합니다.  
- 임의의 3개의 점 a, b, c에 대한 함수 f(a, b, c)의 값을 a와 b 사이의 거리, b와 c 사이의 거리, c와 a 사이의 거리, 3개 값의 중간값으로 정의합니다.  
  
트리의 정점의 개수 n과 트리의 간선을 나타내는 2차원 정수 배열 edges가 매개변수로 주어집니다. 주어진 트리에서 임의의 3개의 점을 뽑아 만들 수 있는 모든 f값 중에서, 제일 큰 값을 구해 return 하도록 solution 함수를 완성해주세요.  
  
#### 제한 조건  
- n은 3 이상 250,000 이하입니다.
- edges의 행의 개수는 n-1 입니다.
    - edges의 각 행은 [v1, v2] 2개의 정수로 이루어져 있으며, 이는 v1번 정점과 v2번 정점 사이에 간선이 있음을 의미합니다.
    - v1, v2는 각각 1 이상 n 이하입니다.
    - v1, v2는 다른 수입니다.
    - 입력으로 주어지는 그래프는 항상 트리입니다.  
  
#### 입출력 예  
|n|	edges|	result|
|:---:|:---:|:---:|
|4|	[[1,2],[2,3],[3,4]]|	2|
|5|	[[1,5],[2,5],[3,5],[4,5]]|	2|

---

### 문제 풀이

트리의 지름을 $D$라고 할 때, 답은 최소 $D−1$ 이상입니다. 왜냐하면, 트리의 지름의 양 끝 두 점 $a$, $b$을 고르고 나머지 한 점 $c$는 $a$ 또는 $b$에 인접한 점으로 선택한다면, $f(a,b,c)=D−1$이 되기 때문입니다.$f(a,b,c)$는 $D$보다 커질 수 없으므로, $f(a,b,c)=D$가 가능한지 불가능한지만 판별하면 됩니다. 
$f(a,b,c)=D$ 이려면, 트리의 지름으로 가능한 경로가 트리 상에서 최소 2개 이상 있어야 합니다. 트리의 지름 경로가 여러 개인지 판별하는 방법은 다음과 같습니다.

1. 임의의 한 점 $v_1$을 정한다.
2. $v_1$에서 제일 먼 점 $v_2$을 찾는다. 이때, 가능한 후보는 여러 개가 나올 수 있다. 그러면 그냥 아무거나 찍는다.
3. 다시 $v_2$에서 제일 먼 점 $v_3$을 찾는다. 이때, 가능한 후보는 여러 개가 나올 수 있다. 그렇다면 트리의 지름으로 가능한 경로가 여러 개인 것이고, 그냥 여기서 종료한다.
4. 다시 $v_3$에서 제일 먼 점 $v_4$를 찾는다. 이때, 가능한 후보가 여러 개라면 지름 경로도 여러 개이고, 아니면 그냥 1개이다.  
  
이렇게 해도 풀리는 이유는, 모든 트리의 지름은 트리의 중앙 지점(Centroid)를 지나기 때문입니다. Centroid를 기준으로 케이스 분류를 해보신다면 답이 보이실 겁니다. 시간복잡도는 $O(n)$ 입니다.

In [1]:
from collections import defaultdict
from queue import Queue

def check_distance(n, adjacent, a, before=None):
    q = Queue() # 큐
    checked = [False for _ in range(n)]
    checked[a] = True
    for idx, dis in list(adjacent[a].items()): # a번 노드와 연결 거리가 확인된 노드에 대해서
        q.put((idx, dis)) # (노드인덱스, a번 노드와의 거리) 정보 push
        checked[idx] = True
        
    far_val = [] # a번 노드와 가장 먼 노드들
    max_dis = 0 # a번 노드와의 최대 거리
    while not q.empty():
        idx, dis = q.get() # (노드인덱스, a번 노드와의 거리)정보를 큐에서 pop
        
        if dis > max_dis: # 최대 거리보다 클 때
            far_val = [idx] # 최대 거리 노드 인덱스 저장
            max_dis = dis # 최대 거리 갱신
        elif dis == max_dis: # 현재의 최대 거리와 같을 때
            far_val.append(idx) # 최대 거리 노드 인덱스 저장
            
        for i, d in list(adjacent[idx].items()): # 현재 탐색중인 노드와 연결 거리가 확인된 노드에 대해서
            if checked[i]:
                continue
            checked[i] = True
            adjacent[a][i] = dis+d # a와 i번 노드와의 거리 갱신
            q.put((i, dis+d)) # (노드 인덱스, a번 노드와의 거리) 정보를 큐에 push
    
    if len(far_val) > 1: # 만약 a번 노드와의 거리가 최대인 노드가 2개 이상일 때 (a와 b,c -> [a,b] == [a,c] > [b,c])
        return max_dis # 최대 거리 반환
    if far_val[0] == before: # 노드 2개가 서로에게는 최대 거리의 노드가 될 때 (a와 b -> [a,b] > [a, 임의의 c], [b, 임의의 c])
        return max_dis-1 # 노드의 지름 - 1 값 반환
    # 위의 상황을 제외하고 최대 거리의 노드가 1개일 경우 해당 노드에 대해서 다시 상대 노드와의 거리를 체크
    return check_distance(n, adjacent, far_val[0], a) 
        
    
def solution(n, edges):
    answer = 0 # 답
    adjacent = [defaultdict(int) for _ in range(n)] # 해당 노드 사이의 거리
    for a,b in edges: # 주어진 값에 대해서
        adjacent[a-1][b-1] = 1
        adjacent[b-1][a-1] = 1
        # 노드 사이의 거리를 갱신해줌
    q = Queue() # 큐
    checked = [False for _ in range(n)]
    checked[0] = True
    for i, d in list(adjacent[0].items()): # 0번 노드와 연결거리가 확인된 노드에 대해서
        q.put((i, d)) # 큐에 (노드 인덱스, 거리) 정보를 push
        checked[i] = True
    start = 0 # 탐색 시작할 노드 인덱스 (0번 노드와 가장 거리가 먼 노드)
    max_dis = 0 # 최대 거리
    while not q.empty():
        idx, dis = q.get() # (노드 인덱스, 거리) 정보 pop
        if dis > max_dis: # 지금까지 최대 거리일 때
            start = idx # 탐색 시작할 노드 인덱스 갱신
            max_dis = dis # 최대 거리 갱신
        for i, d in list(adjacent[idx].items()): # 현재 탐색 중인 노드와 연결거리가 확인된 노드에 대해서
            if checked[i]:
                continue
            checked[i] = True
            q.put((i, d+dis)) # (노드 인덱스, 0번 노드와의 거리) 정보를 push
    answer = check_distance(n, adjacent, start) # start번 노드에 대해서 중간값의 최대값 탐색
    return answer # 답 반환

In [2]:
n = 4
edges = [[1,2],[2,3],[3,4]]
solution(n, edges)

2

In [3]:
n = 5
edges = [[1,5],[2,5],[3,5],[4,5]]
solution(n, edges)

2